# Визуализация.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv('/kaggle/input/cardataset/data.csv')

In [ ]:
data.shape

In [ ]:
data.head()

Переведём в понятные единицы измерения.

In [ ]:
data['highway l/km'] = round(235.214583 / data['highway MPG'], 2)
data['city l/km'] = round(235.214583 / data['city mpg'], 2)
data.drop(columns=['highway MPG', 'city mpg'], inplace=True)
data.head()

Сделаем некоторые значения категориальными (количество цилиндров, год, количество дверей)

In [ ]:
data[['Year', 'Engine Cylinders', 'Number of Doors']] = data[['Year', 'Engine Cylinders', 'Number of Doors']].astype('category')

In [ ]:
data[['Make', 'Model', 'Engine Fuel Type', 'Transmission Type', 'Driven_Wheels', 'Market Category', 'Vehicle Size', 'Vehicle Style', 'Number of Doors', 'Engine Cylinders', 'Year']].describe()

### Посмотрим на числовые признаки

#### Мощность

In [ ]:
sns.distplot(data['Engine HP'].dropna(), bins=73).set_title('Распредедление мощности автомобилей в л. с. ')

Интересно, что на графике распределений лошадиных сил два пика и просадка в районе 250 л. с. Вероятно, это связано с налогообложением в зависимости от мощности автомобиля. 

#### MSRP

In [ ]:
sns.distplot(data['MSRP'])

In [ ]:
sns.distplot(data[data['MSRP']<150000]['MSRP']).set_title('Распредедление рекомендованной цены производителя')

Распределение для MSRP (не для всех, а для меньших 150000, так как иначе картинка получается неинформативной)<br>
(MSRP - рекомендованная цена производителя)<br>
Заметно, что рядом с нулём есть всплеск, рассмотрим его.   

In [ ]:
sns.distplot(data[data['MSRP']<40000]['MSRP']).set_title('Распредедление рекомендованной цены производителя')

In [ ]:
data['MSRP'].value_counts()

In [ ]:
data[data['MSRP'] == 2000]['Year'].value_counts()

Возможно, 2000 - это какой-то минимум для цены. Только машины до 2000-го года оценивают этим минимумом.

#### Популярность

In [ ]:
sns.distplot(data['Popularity'], bins=10).set_title('Распредедление популярности')

#### Расход топлива

In [ ]:
sns.distplot(data['city l/km'], label='В городе')
sns.distplot(data['highway l/km'], label='На трассе')
plt.legend()
plt.suptitle('Распределение расхода топлива на 100 км в городе и на трассе')

### Посмотрим на категориальные признаки

In [ ]:
sns.set(style='whitegrid')
qualitative_colors = sns.cubehelix_palette(8)

#### Количество цилиндров

In [ ]:
sns.countplot(data['Engine Cylinders'], color=qualitative_colors[2])
plt.suptitle('Количество цилиндров')

#### Количество дверей

In [ ]:
sns.countplot(data['Number of Doors'], color=qualitative_colors[1])
plt.suptitle('Количество дверей')

#### Тип трансмиссии

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
sns.countplot(data['Transmission Type'], color=qualitative_colors[0], ax=ax)
plt.suptitle('Трансмиссия')

Неудивительно, что трансмиссий типа Direct Drive так немного, ведь это новая разработка. А автоматических коробок больше всего, они сейчас самые популярные. <br>
Видно, что есть пропуски в данных. 

#### Тип ТС

In [ ]:
sns.set()
sns.set(font_scale=1.4)
fig, ax = plt.subplots(figsize=(38, 8))
sns.countplot(data['Vehicle Style'], color=qualitative_colors[3], ax=ax)
plt.suptitle('Типы')
sns.set()

#### Размер ТС

In [ ]:
sns.countplot(data['Vehicle Size'], color=qualitative_colors[4])
plt.suptitle('Размеры')

#### Ведущие колёса

In [ ]:
splot = sns.countplot(data['Driven_Wheels'], color=qualitative_colors[5])
plt.suptitle('Ведущие колёса')

Четыре ведущих колеса = все колёса ведущие. Можем подправить датасет. 

In [ ]:
data['Driven_Wheels'] = data['Driven_Wheels'].apply(lambda x: 'all wheel drive' if x == 'four wheel drive' else x).astype('category')

In [ ]:
sns.countplot(data['Driven_Wheels'], color=qualitative_colors[5])
plt.suptitle('Ведущие колёса')

#### Тип топлива

In [ ]:
data['Engine Fuel Type'].value_counts()

#### Посмотрим табличку с производителями. Их слишком много для гистограмы. 

In [ ]:
data['Make'].value_counts()

### Посмотрим зависимости между некоторыми признаками

In [ ]:
sns.set()

#### Зависимость мощности от количества цилиндров

In [ ]:
sns.swarmplot(data['Engine Cylinders'], data['Engine HP'])

In [ ]:
sns.barplot(data['Engine Cylinders'], data['Engine HP'])

В целом, чем больше цилиндров, тем больше мощность автомобиля.<br>
0 цилиндров скорее всвего соответствует электрическим автомобилям. Проверим это:

In [ ]:
len(data[(data['Engine Cylinders'] == 0) & (data['Engine Fuel Type'] != 'electric')])

Так и есть!

#### Зависимость MSRP от мощности

In [ ]:
sns.scatterplot(data['Engine HP'], data['MSRP'])
plt.suptitle('Зависимость MSRP от мощности')

Видно, что есть несколько очень дорогих и очень мощных автомобилей. И несколько не таких мощных, но всё равно дорогих. Посмотрим, что это за машины. 

In [ ]:
data[data['MSRP'] > 1000000]

Обрежем по MSRP, чтобы было лучше видно. 

In [ ]:
sns.scatterplot(data[data['MSRP'] < 900000]['Engine HP'], data[data['MSRP'] < 900000]['MSRP'])
plt.suptitle('Зависимость MSRP от мощности')

Заменто, что при одинаковой мощности цена может очень отличаться. Видны даже вертикальные линии. 
Общая тенденция - больше мощность - больше цена. 

#### Зависимость MSRP от размера

Swarmplot для зависимости MSRP от размера работал очень долго. Построим вместо него barplot. Он, конечно, не даст полной картины, но общая закономерность будет видна. 

In [ ]:
sns.barplot(data['Vehicle Size'], data['MSRP'])
plt.suptitle('Зависимость MSRP от размера')

В среднем, чем машина больше, тем она дороже. 

#### Зависимость MSRP от производителя

In [ ]:
sns.set()
a = data.groupby(['Make'])['MSRP'].mean().sort_values()
Maker_cheap = list(a[:3].index)
Maker_expensive = list(a[-3:].index)
sns.set(font_scale=1.5)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
sns.swarmplot(data[data['Make'].isin(Maker_cheap)]['Make'], data[data['Make'].isin(Maker_cheap)]['MSRP'], ax=ax1)
sns.swarmplot(data[data['Make'].isin(Maker_expensive)]['Make'], data[data['Make'].isin(Maker_expensive)]['MSRP'], ax=ax2)
ax1.set_title('Зависимость MSRP от производителя\n(3 самых дешёвых в среднем)')
ax2.set_title('Зависимость MSRP от производителя\n(3 самых дорогих в среднем)')
sns.set()

#### Зависимость расхода от мощности

In [ ]:
sns.scatterplot(data['Engine HP'], data['city l/km'])
plt.suptitle('Зависимость расхода в городе от мощности')

In [ ]:
sns.scatterplot(data['Engine HP'], data['highway l/km'])
plt.suptitle('Зависимость расхода на трассе от мощности')

In [ ]:
sns.scatterplot(data['Engine HP'], data['city l/km'], label='В городе')
sns.scatterplot(data['Engine HP'], data['highway l/km'], label='На трассе')
plt.legend()
plt.suptitle('Зависимость расхода от мощности')

Видны горизонтальные линии. Я думала, что расход будет коррелировать с мощностью, на деле связь не так уж видна. 